<b>Station calibration</b>

This script pushes a temperature change from the field station to the station calibrator then gets a calibration solution from the calibration store.

At present the GetSolution command on the stub calibration store returns a list of numbers with the temperature and channel prepended to it.


Setup

In [ ]:
import tango
import json

field_station = tango.DeviceProxy("low-mccs/mockfieldstation/001")
station_calibrator = tango.DeviceProxy("low-mccs/stationcalibrator/001")
calibration_store = tango.DeviceProxy("low-mccs/calibrationstore/001")
for device in [field_station, calibration_store, station_calibrator]:
    device.adminMode = 0

Store solutions in calibration store

In [ ]:
for outside_temperature in [25 + (0.5 * i) for i in range(40)]:
    for frequency_channel in range(10):
        solution = [1.0 + frequency_channel % 2] + [0.5 * i * (outside_temperature % 17) for i in range(8)]
        args = json.dumps({"outside_temperature": outside_temperature, "frequency_channel": frequency_channel, "solution": solution})
        calibration_store.StoreSolution(args)

Push change in temperature

In [ ]:
field_station.MockOutsideTemperatureChange(27.5)

Get calibration solution using Station Calibrator

In [ ]:
station_calibrator.GetCalibration(json.dumps({"frequency_channel": 3}))

Push another change in temperature

In [ ]:
field_station.MockOutsideTemperatureChange(42.0)

Get new calibration solution using Station Calibrator

In [ ]:
station_calibrator.GetCalibration(json.dumps({"frequency_channel": 4}))